In [1]:
from pysat.formula import CNF
import more_itertools as mit
from graphviz import Digraph
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import HTML

In [24]:
EXPLANATION = """\
<div class="app-sidebar">

</div>
"""

In [25]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [1]:
class App:

    def __init__(self,cnf_value):
        
        self._cnf_value=cnf_value
        self._cnf_txtarea=self._create_cnf_txtarea()
        self._marco_output_txtarea=self._create_marco_output_txtarea(cnf_value)
        
        self._plot_container = widgets.Output()
        
        
        
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '''<h1>Dashboard for MARCO</h1>
                    <p>This demo demonstrates how the MARCO algorithm works on <a href="https://people.sc.fsu.edu/~jburkardt/data/cnf/cnf.html">CNF files</a>.
                    In this dashboard, you can change the CNF input file content and the MARCO output as well as the following visualization will change accordingly'''
                ), 
                layout=widgets.Layout(margin='0 0 1em 0')
            ),
            widgets.HTML(
                (
                    '''<h3>Section1 - Input and Output</h3>
                    <p>In this section, we mainly focus on design a way for users to simplily modify the input and make it visible with following steps within one interface'''
                ), 
                layout=widgets.Layout(margin='0 0 1em 0')
            ),
            
            widgets.HBox([widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 1em'))]),
            
            
            
            
            widgets.HBox([self._cnf_txtarea, self._marco_output_txtarea]),
            
            widgets.HTML(
                (
                    '''<h3>Section2 - Visualization</h3>
                    <p> In order to visualize the MSSes and MUSes, firstly we generate a powerset based on the totla number of clauses in CNF file and then extract MUSes and MSSes.<br>
                    After that, the code will figure out all MUSes, mark them as dark red and mark the supersets as lightred meanwhile all MSSes will be marked as dark green and their subset would be lightgreen
                    '''
                ), 
                layout=widgets.Layout(margin='0 0 1em 0')
            ),
            
            widgets.HBox([self._plot_container]),
           
        ], 
            layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        
        self._update_app()
        
        
    @classmethod
    def load_cnf(cls,cnf_value):
        return cls(cnf_value)
    
    def _create_cnf_txtarea(self):
        cnf_txtarea=widgets.Textarea(value='p cnf 2 5\n1 0\n-1 0\n2 0\n-2 0 \n1 2 0',description='CNF File:',disabled=False,layout=Layout(width='400px',height='350px'))
        cnf_txtarea.observe(self._on_change, names=['value'])
        return cnf_txtarea
    
    
    def _create_marco_output_txtarea(self,cnf_value):
        formula=CNF(from_string=cnf_value)
        formula.to_file('formula.cnf')
        MARCO_output=! MARCO/marco.py formula.cnf -v
        string=''
        for output_line in MARCO_output:
            string=string+output_line+'\n'
        style={'description_width':'150px'}
        marco_output_txtarea=widgets.Textarea(value=string,description='MARCO output:',disabled=False,layout=Layout(width='400px',height='350px'),style=style)
        return marco_output_txtarea
        
    def _create_plot(self,marco_output_value,cnf_value): 
        formula=CNF(from_string=cnf_value)
        N=len(formula.clauses)
        MARCO_output=! MARCO/marco.py formula.cnf -v
        power_set=[set(item) for item in list(mit.powerset([item for item in range(1,N+1)]))]
        MSSes=[]
        MUSes=[]
        for s in MARCO_output:
            myinput = s[2:]
            myset = set(map(int,myinput.split(" ")))
            if s[0]=="U":
                MUSes.append(myset)
            else:
                MSSes.append(myset)
        pg=Digraph(format='png') #pg refer to powerset_graph
        pg.attr(rankdir='BT')
        for tup_x in power_set:
            for MUS in MUSes:
                if tup_x.issuperset(MUS):
                    pg.node(str(tup_x),style="filled",color="#fb9a99")
            for MSS in MSSes:
                if tup_x.issubset(MSS):
                    pg.node(str(tup_x),style="filled",color="#b2dfba")
            if tup_x in MSSes:
                pg.node(str(tup_x),style="filled",color="green3")
            if tup_x in MUSes:
                pg.node(str(tup_x),style="filled",color="red")
            for tup_y in power_set:
                if set(tup_x).issuperset(set(tup_y)) and len(tup_y)==len(tup_x)-1:
                    pg.edge(str(tup_y),str(tup_x))
        pg.render(filename='tmp')
        im= open("tmp.png", "rb").read()
        image_widget=widgets.Image(value=im, format='png',width='auto',height='auto')
        return(image_widget)
    
    
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        cnf_value=self._cnf_txtarea.value
        marco_output_value=self._create_marco_output_txtarea(cnf_value).value
        self._marco_output_txtarea.value=self._create_marco_output_txtarea(cnf_value).value

        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            display(self._create_plot(marco_output_value,cnf_value))

In [2]:
CNF_value='p cnf 2 5\n1 0\n-1 0\n2 0\n-2 0 \n1 2 0'
app = App.load_cnf(CNF_value)
app.container

NameError: name 'widgets' is not defined